In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import keras.layers as L
from keras.models import Model,Sequential
import math
from keras.utils import Sequence
from keras.preprocessing import image
from random import shuffle
from sklearn.model_selection import train_test_split
import plotly.express as px
import seaborn as sns
from sklearn.utils import class_weight
from librosa import cqt
from skimage.transform import resize
from random import shuffle
import glob
import pathlib
import cv2
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
train_labels =pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
sample_submission = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

In [ ]:
colors=("black", "red", "green")
signal_names=("LIGO Hanford", "LIGO Livingston", "Virgo")
x = np.load('../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy').astype(np.float32)
fig = plt.figure(figsize=(16, 10))

    
plt.subplot(4, 1, 4)
plt.plot(x)
plt.legend(signal_names, fontsize=12, loc="lower right")

In [ ]:
def id_to_path(idx,train=True):
    path = '../input/g2net-gravitational-wave-detection/'
    if train:
        folder = 'train/'
    else:
        folder = 'test/'
    path+=folder+idx[0]+'/'+idx[1]+'/'+idx[2]+'/'+idx+'.npy'
    return path

In [ ]:
def id_to_path2(idx,train=True):
    path = '../input/g2net-constant-q-transformed/'
    if train:
        folder = 'train_cqt/train_cqt/'
    else:
        folder = 'test_cqt/test_cqt/'
    path+=folder+idx+'.png'
    return path

In [ ]:
def getimg(x):
    cq=np.hstack(x)
    cq=cq/np.max(cq)
    cq = cqt(cq,sr=2048, fmin=20, n_bins=64,pad_mode='wrap')
    return cq

In [ ]:
img = getimg(x)
img.shape

In [ ]:
#img = resize(abs(img), (65,32))
img = img.reshape(50,32)
img2= np.stack([img,img,img],axis = -1)
img2.shape

In [ ]:
plt.imshow(abs(img2),aspect='auto', origin='lower')
plt.xlabel('Time Steps')
plt.ylabel('Freq bins')

In [ ]:
plt.imshow(abs(img2),aspect='auto', origin='lower')
plt.xlabel('Time Steps')
plt.ylabel('Freq bins')

In [ ]:
img = cv2.imread('../input/g2net-constant-q-transformed/train_cqt/train_cqt/00000e74ad.png')
plt.imshow(img)

In [ ]:
img = cv2.imread(id_to_path2('00000e74ad'))
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
class Dataset(Sequence):
    def __init__(self,idx,directory,batch_size=16,shuffle=True):
        self.idx = idx
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.directory = directory
        self.is_train = True
        self.y = []
        if self.directory == 'test':
            self.is_train = False
        
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
    def getimg(self,x):
        cq=cv2.imread(x)
        return cq
    def getlabel(self,x):
        x=x.split('/')[4]
        if x == 'ones':
            return 1
        else:
            return 0
                  
    def __getitem__(self,ids):
        batch_ids = self.idx[ids * self.batch_size:(ids + 1) * self.batch_size]    
        list_x1 = np.array([self.getimg(x) for x in batch_ids])
        if self.directory != 'test':
            batch_y = np.array([self.getlabel(x) for x in batch_ids])
            return list_x1, batch_y
        else:
            return list_x1

In [ ]:
train_ones = [file for file in sorted(glob.glob("../input/g2net-as-image/train/ones/*"))]
train_zero = [file for file in sorted(glob.glob("../input/g2net-as-image/train/zero/*"))]
train = train_ones+train_zero
shuffle(train)

In [ ]:
valid_ones = [file for file in sorted(glob.glob("../input/g2net-as-image/valid/ones/*"))]
valid_zero = [file for file in sorted(glob.glob("../input/g2net-as-image/valid/zero/*"))]
valid = valid_ones + valid_zero
shuffle(valid)

In [ ]:
train_data = train + valid
shuffle(train_data)

In [ ]:
test_idx = [file for file in sorted(glob.glob("../input/g2net-as-image/test/*"))]

In [ ]:
pip install google-cloud-automl

In [ ]:
!pip install -U efficientnet

In [ ]:
import efficientnet.keras as efn

In [ ]:
ef=efn.EfficientNetB0(input_shape=(67, 193, 3),weights='imagenet',include_top=False)

In [ ]:
#for layer in ef.layers[:]:
#    layer.trainable = False


In [ ]:
pip install tensorflow-hub

In [ ]:
import tensorflow_hub as hub

In [ ]:
hub_url = 'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l/feature-vector'

In [ ]:
def crt_model():
    inp=L.Input(shape=(67, 193, 3))
    l = ef(inp)
    #l = hub.KerasLayer(hub_url, trainable=True)(inp)
    #l = L.Conv2D(64,(3,3), activation = 'relu')(inp)
    l = L.Flatten()(l)
    l = L.Dense(1, activation='sigmoid')(l)
    model = Model(inputs= [inp],outputs=l)
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=[keras.metrics.AUC()])
    return model

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for k, (train_idx, valid_idx) in enumerate(kf.split(train_data)):
    print(train_idx, end=" ")
    print(valid_idx)


In [ ]:
def get_res():
    nfolds = 4
    scores =[]
    preds = np.zeros((len(test_idx)))
    kf = KFold(n_splits=nfolds, shuffle=True, random_state=42)
    for k, (train_idx, valid_idx) in enumerate(kf.split(train_data)): 
        model=crt_model()
        train_datas = np.array(train_data)
        train_x,test_x=train_datas[train_idx], train_datas[valid_idx]      
        train_dataset = Dataset(train_x,directory = 'train')
        valid_dataset = Dataset(test_x,directory = 'valid', batch_size=64)
        test_dataset  = Dataset(test_idx,directory = 'test', batch_size=64)
        history = model.fit(train_dataset,epochs=2,validation_data=valid_dataset)
        y_preds = model.predict(test_dataset)
        y_preds = y_preds.reshape(-1)
        
        preds+=y_preds
    return np.array(preds)/nfolds 

In [ ]:
preds = get_res()

In [ ]:
preds = preds.reshape(-1)

In [ ]:
test_samples = []

In [ ]:
def getid(path):
    x = path.split('/')[4]
    x = x.split('.')[0]
    return x

In [ ]:
for i in test_idx:
    test_samples.append(getid(i))

In [ ]:
len(test_samples)

In [ ]:
submission = pd.DataFrame({'id':test_samples,'target':preds})
submission.to_csv('submission.csv',index=False)

In [ ]:
submission